# Network Update 2023

Its that time of the year again, lets see how many things needs to be changed this time.

In [2]:
from psycopg2 import connect
from psycopg2 import sql
from psycopg2.extras import execute_values
from pathlib import Path
import pandas as pd
import pandas.io.sql as pandasql
import configparser
import matplotlib.pyplot as plt
CONFIG = configparser.ConfigParser()
CONFIG.read(str(Path.home().joinpath('db.cfg')))
dbset = CONFIG['DBSETTINGS']
con = connect(**dbset)

## Number of nodes that needs to be updated
only two!

In [4]:
sql = '''
select node_id from congestion.network_nodes
except 
select node_id from here.routing_nodes_23_4 
    '''
with con : 
    nodes = pandasql.read_sql(sql, con) 
print(nodes)   

/tmp/ipykernel_3548274/1008659884.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  nodes = pandasql.read_sql(sql, con)


      node_id
0  30420390.0
1  30420392.0


## Number of links that needs to be updated
2990 links needs to be retired omg

In [5]:
sql = '''
select link_dir from congestion.network_links_22_2
except 
select link_dir from here.routing_streets_23_4 
    '''
with con : 
    nodes = pandasql.read_sql(sql, con) 
print(nodes)   

         link_dir
0     1207662665F
1     1258933853T
2     1000005503T
3      995198819F
4       29568114T
...           ...
2985   949289551T
2986  1258735603F
2987  1258732905T
2988   993964936T
2989  1258776870T

[2990 rows x 1 columns]


/tmp/ipykernel_3548274/580113570.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  nodes = pandasql.read_sql(sql, con)


## Number of segments that needs to be updated
1812 segment_ids needs be to retired and route............

In [6]:
sql = '''
with changed_links AS (
	select link_dir from congestion.network_links_22_2
	except 
	select link_dir from here.routing_streets_23_4)
	
select distinct segment_id from  congestion.network_links_22_2 
inner join changed_links using (link_dir)
    '''
with con : 
    nodes = pandasql.read_sql(sql, con) 
print(nodes)   

      segment_id
0           6062
1           2335
2            652
3           5856
4           1560
...          ...
1807        2359
1808        3795
1809        1715
1810        3697
1811        4035

[1812 rows x 1 columns]


/tmp/ipykernel_3548274/3611360751.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  nodes = pandasql.read_sql(sql, con)


## Number of new and retired traffic signals
Quite a bunch of new traffic signals. We can deal with this after we recreated the network. 

In [9]:
sql = '''
select count(1) from gis.traffic_signal
where activationdate  >= '2022-01-01'
    '''
with con : 
    nodes = pandasql.read_sql(sql, con) 
nodes.head()   

/tmp/ipykernel_3548274/3409494311.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  nodes = pandasql.read_sql(sql, con)


,count
0,43


# Updating the links

In [ ]:
'''
-- Create new links table except the segment_ids that are outdated

CREATE TABLE congestion.network_links_23_4 AS 
-- retired links
with changed_links AS (
	select link_dir from congestion.network_links_22_2
	except 
	select link_dir from here.routing_streets_23_4)
-- retired segments	
, changed_seg AS (
	select distinct segment_id, start_vid, end_vid 
	from  congestion.network_links_22_2
	inner join changed_links using (link_dir))
-- everything in network_links other than the retired segments
select      network_links_22_2.* 
from 		congestion.network_links_22_2
left join 	changed_seg using (segment_id)
where 		changed_seg is null ;

'''
'''
-- Insert routed results using new map version (23_4) for retired segments

with changed_links AS (
	select link_dir from congestion.network_links_22_2
	except 
	select link_dir from here.routing_streets_23_4)	
, changed_seg AS (
	select distinct segment_id, start_vid, end_vid 
	from  congestion.network_links_22_2 
	inner join changed_links using (link_dir))
 
 -- insert result to newly created network_links table
INSERT INTO congestion.network_links_23_4
SELECT segment_id, start_vid, end_vid, link_dir, routing.geom, round(st_length(st_transform(geom, 2952))::numeric, 2) as length
FROM   changed_seg
CROSS JOIN LATERAL pgr_dijkstra('SELECT id, source::int, target::int, st_length(st_transform(geom, 2952)) as cost 
				 	   			FROM here.routing_streets_23_4',  -- route using new map version's routing_streets
								start_vid, 
								end_vid)
INNER JOIN here.routing_streets_23_4 routing ON id = edge;
''';

## Check to see if all segments are inserted 
Returns nothing! 

In [13]:
sql = '''
select distinct segment_id from congestion.network_links_22_2
except
select distinct segment_id from congestion.network_links_23_4
    '''
with con : 
    nodes = pandasql.read_sql(sql, con) 
nodes.head()  

/tmp/ipykernel_3548274/647985274.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  nodes = pandasql.read_sql(sql, con)


,segment_id


## Check to see if both returned the same number of segment_id
Both have 6558 segments. 

In [15]:
sql = '''

select count(distinct segment_id) from congestion.network_links_22_2
    '''
with con : 
    nodes = pandasql.read_sql(sql, con) 
nodes.head()  

/tmp/ipykernel_3548274/1608784709.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  nodes = pandasql.read_sql(sql, con)


,count
0,6558


In [16]:
sql = '''
select count(distinct segment_id) from congestion.network_links_23_4
    '''
with con : 
    nodes = pandasql.read_sql(sql, con) 
nodes.head()  

/tmp/ipykernel_3548274/2532337583.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  nodes = pandasql.read_sql(sql, con)


,count
0,6558


## Finding if there are new traffic signals to add in 
25?!?!
Check them out on QGIS and see if they are legit. 

In [21]:
sql = '''
-- Find segments that needs to be updated due to new traffic signals
with new_signal as (
	select px, ST_Transform(ST_buffer(ST_Transform(geom, 2952), 50), 4326) as geom 
	from gis.traffic_signal
	left join congestion.network_int_px_22_2 a using (px)
	where activationdate >= '2022-04-17' and a.px is null )

select count(distinct px)
from congestion.network_segments seg
join new_signal on ST_intersects(new_signal.geom, seg.geom)
    '''
with con : 
    nodes = pandasql.read_sql(sql, con) 
nodes.head()  

/tmp/ipykernel_3548274/690658099.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  nodes = pandasql.read_sql(sql, con)


,count
0,25


## Find closest nodes for these new traffic signals 

Checked on QGIS, mostly look good in distance difference, except for one that has 40m difference. 

In [22]:
'''
-- Find segments that needs to be updated due to new traffic signals
with new_signal as (
	select px, ST_Transform(ST_buffer(ST_Transform(geom, 2952), 50), 4326) as bgeom, geom 
	from gis.traffic_signal
	left join congestion.network_int_px_22_2 a using (px)
	where activationdate >= '2022-04-17' and a.px is null )

, new_px AS (
	select distinct px, new_signal.geom as px_geom
	from congestion.network_segments seg
	join new_signal on ST_intersects(new_signal.bgeom, seg.geom))

-- Find new nodes for outdated nodes using nearest neighbour
			
select 	node_id, px, geom, px_geom, dist
from 	here.routing_nodes_23_4
CROSS JOIN LATERAL (SELECT z.px,
							px_geom, 
							(ST_transform(geom, 2952) <-> ST_Transform(z.px_geom, 2952)) as dist	
					FROM new_px z
					ORDER BY (geom <-> z.px_geom)
					LIMIT 1) nodes;
''';